# 🤖 実験3：Webアプリ用モデルの作成と保存
*情報保障システム工学・演習 2025 – 第4回*

**本日のゴール:**
1. DNNの学習を実装する
2. 線形モデル、アンサンブル、DNNなど複数のモデルを試し、最適なモデルを選択する
3. Webアプリで読み込める形式 (`model_meta.json`) でモデルを保存する

## 0. ライブラリ読み込み

In [1]:
!pip install scikeras -q
print('✔️ scikeras インストールOK')

✔️ scikeras インストールOK


In [2]:
import pandas as pd, numpy as np, glob, json, time, os
from google.colab import files
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
print('✔️ ライブラリ準備OK')

✔️ ライブラリ準備OK


## 1. CSV を `data/` にアップロード
1. 左のフォルダ ▶ で `右クリック` → **`data`** フォルダ作成  
2. `1-01.csv`〜`9-20.csv` を **ドラッグ&ドロップ**
3. 下セルで 180 ファイルあるか確認

In [13]:
csv_files = sorted(glob.glob('data2/*.csv'))
print('ファイル数:', len(csv_files))
assert len(csv_files)==180, '180 ファイルない場合はアップロード漏れを確認'

ファイル数: 180


## 2. 全データを1つの DataFrame に結合

In [14]:
df_list=[]
for f in csv_files:
    label, num = os.path.splitext(os.path.basename(f))[0].split('-')
    tmp = pd.read_csv(f)
    tmp['label'] = int(label); tmp['num'] = int(num)
    df_list.append(tmp)

total_df = pd.concat(df_list, ignore_index=True)
print('total_df shape:', total_df.shape)
total_df.head()

total_df shape: (3780, 6)


,index,x,y,z,label,num
0,0,0.586765,0.900168,-0.000001,1,1
1,1,0.506538,0.829854,-0.039034,1,1
2,2,0.437750,0.687593,-0.067367,1,1
3,3,0.461338,0.570361,-0.099242,1,1
4,4,0.544183,0.524047,-0.128717,1,1


## 3. 特徴量エンジニアリング

WebアプリのJavaScriptで行う前処理と**全く同じ**ロジックで、特徴量 `X` と正解ラベル `y` を作成します。

In [15]:
def create_normalized_features(df):
    """DataFrameから正規化済みの特徴量Xとラベルyを生成する"""
    X_list, y_list = [], []

    # 'label'と'num'でグループ化し、1サンプル(1つのジェスチャー)ごとに処理
    for (label, num), group in df.groupby(['label', 'num']):
        # ランドマークのインデックス順にソートし、x,y座標をNumpy配列に変換
        lm = group.sort_values('index')[['x', 'y']].to_numpy() # shape(21, 2)

        # ① 手首(0番)を原点に移動（位置の正規化）
        wrist = lm[0].copy() # wristの座標をコピー
        lm -= wrist

        # ② 手の大きさでスケール（大きさの正規化）
        # 0除算を避けるため、ノルムが0の場合は1とする
        norm = np.linalg.norm(lm, axis=1).max() or 1.0
        lm /= norm

        # ③ 42次元ベクトルに変換し、リストに追加
        X_list.append(lm.flatten())
        y_list.append(label)

    return np.array(X_list), np.array(y_list)

# 関数を実行してX, yを作成
X, y = create_normalized_features(total_df)
print('X shape:', X.shape) # (180, 42)のはず
print('y shape:', y.shape) # (180,)のはず

X shape: (180, 42)
y shape: (180,)


## 4. 訓練データとテストデータに分割

In [16]:
# stratify=y を指定し、各クラスの割合が訓練/テストで均等になるように分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (126, 42)
X_test shape: (54, 42)


## 5. モデル比較
様々な種類のモデルで精度と速度を比較します。

In [17]:
def create_dnn_model():
    """比較用のDNNモデルを定義して返す関数"""
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(42,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        # 出力層: 0-9の10クラス分類 (ラベルが1-9でも10クラスで問題ない)
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [18]:
# ===================================================================
# 以下のコードで、モデル比較のセル全体を置き換えてください
# ===================================================================

# --- 1. scikit-learn系統のモデルで学習と評価 ---
models_sklearn = {
    # ロジスティック回帰
    "LogReg": Pipeline([
        ('sc', StandardScaler()),
        ('clf', LogisticRegression(max_iter=1000))
    ]),
    # 線形SVM (liblinearベース、高速)
    "LinearSVC": Pipeline([
        ('sc', StandardScaler()),
        ('clf', LinearSVC(C=10, max_iter=2000, random_state=42, dual=False))
    ]),
    # 線形SVM (libsvmベース、OvR戦略を明示)
    "SVC_OvR": Pipeline([
        ('sc', StandardScaler()),
        ('clf', OneVsRestClassifier(SVC(kernel="linear", C=10, probability=True)))
    ]),
    # ランダムフォレスト
    "RandomForest": RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42),
}

results = {}
print("scikit-learnモデルで学習と評価を実行します...")
for name, model in models_sklearn.items():
    t0 = time.time()
    model.fit(X_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_test))
    results[name] = {'accuracy': acc, 'time': time.time() - t0, 'model': model}
    print(f"{name:<15} | accuracy: {acc:.3f}, time: {results[name]['time']:.3f} s")

scikit-learnモデルで学習と評価を実行します...
LogReg          | accuracy: 0.981, time: 0.026 s
LinearSVC       | accuracy: 1.000, time: 0.014 s
SVC_OvR         | accuracy: 1.000, time: 0.056 s
RandomForest    | accuracy: 1.000, time: 0.427 s


In [19]:
# --- 2. DNNモデルの学習と評価 (手動実行) ---
print("\nDNNモデルで学習と評価を手動で実行します...")
name = "DNN"
t0 = time.time()

# (A) データの標準化を手動で実行
scaler_dnn = StandardScaler()
# TODO-1: 訓練データ(X_train)を、scaler_dnnに学習させて(fit)変換(transform)しよう
X_train_scaled = scaler_dnn.fit_transform(X_train) # ← (A) この括弧の中身を考えよう

# テストデータ(X_test)は、訓練データで学習したスケーラーを使って変換するだけ（情報が漏れないようにするため）
X_test_scaled = scaler_dnn.transform(X_test) # ← (B) この括弧の中身を考えよう

# (B) Kerasモデルを直接作成・学習
dnn_model = create_dnn_model()
dnn_model.fit(X_train_scaled, y_train, epochs=60, batch_size=16, verbose=0)

# (C) 予測を行い、確率からクラスラベルに変換
y_pred_probs = dnn_model.predict(X_test_scaled, verbose=0)
y_pred_dnn = np.argmax(y_pred_probs, axis=1) # 最も確率の高いクラスを選ぶ処理

# (D) 精度を計算
# TODO-3: 正しい変数を使って、モデルの精度を計算しよう
# accuracy_score(正解のラベル, モデルの予測結果)
acc_dnn = accuracy_score(y_test, y_pred_dnn) # ← (C) この括弧の中身を考よう

# (E) 結果をresultsに追加
results[name] = {'accuracy': acc_dnn, 'time': time.time() - t0, 'model': dnn_model}
print(f"{name:<15} | accuracy: {acc_dnn:.3f}, time: {results[name]['time']:.3f} s")


DNNモデルで学習と評価を手動で実行します...
DNN             | accuracy: 0.981, time: 8.131 s


In [20]:
# --- 3. 最終的なベストモデルを選択 ---
best_name = max(results, key=lambda k: results[k]['accuracy'])
print(f"\n✅ Best Model: {best_name}")


✅ Best Model: LinearSVC


## 6. 最適モデルの保存
選択したモデルを、Webアプリで読み込めるJSON形式で保存します。

★現在のWebアプリは、OvR形式にしか対応していません

★LinearSVC がベストモデル（accuray がほぼ 1.0）になるはずです！

In [21]:
# ## 6. 最適モデルの保存 のセル

best_name = max(results, key=lambda k: results[k]['accuracy'])
best_model = results[best_name]['model']
print(f"\n✅ Best Model: {best_name}")

# --- 保存ロジック ---
meta = None # 保存するパラメータ
can_export_for_webapp = False # Webアプリ用にエクスポート可能か

if best_name == "DNN":
    print(f"'{best_name}' が最適モデルですが、Webアプリ用のJSON形式では保存できません。")
    print("Webで利用するには、TensorFlow.js形式で別途保存する必要があります。")

elif isinstance(best_model, Pipeline):
    clf = best_model.named_steps.get('clf')
    scaler = best_model.named_steps.get('sc')

    # Webアプリ用のパラメータを抽出できるかチェック
    if hasattr(clf, 'coef_') and hasattr(clf, 'intercept_'):
        # 【追加】coef_の形状をチェックし、OvR戦略のモデルであるか確認
        if clf.coef_.shape[0] == len(set(y)):
            meta = {
                "name":      best_name,
                "accuracy":  results[best_name]['accuracy'],
                "coef":      clf.coef_.tolist(),
                "intercept": clf.intercept_.tolist(),
                "labels":    [int(label) for label in sorted(list(set(y)))],
                "mean":      scaler.mean_.tolist(),
                "scale":     scaler.scale_.tolist()
            }
            can_export_for_webapp = True
            print(f"'{best_name}' のパラメータを抽出しました (OvR形式)。")
        else:
            print(f"'{best_name}' はOvO戦略のため、WebアプリのOvR形式と互換性がありません。")
    else:
        print(f"'{best_name}' はcoef_/intercept_を持たないため、JSON保存に対応していません。")
else:
    # RandomForestのようなPipelineでないモデルの場合
    print(f"'{best_name}' はWebアプリ用のパラメータ抽出に対応していません。")


# JSONファイルとして保存・ダウンロード
if can_export_for_webapp and meta:
    with open('model_meta.json', 'w') as f:
        json.dump(meta, f, indent=2)
    print("\n'model_meta.json' を保存しました。ダウンロードします。")
    from google.colab import files
    files.download('model_meta.json')
else:
    print("\nWebアプリ用のJSONファイルは生成されませんでした。")


✅ Best Model: LinearSVC
'LinearSVC' のパラメータを抽出しました (OvR形式)。

'model_meta.json' を保存しました。ダウンロードします。


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>